# Automated Model Reduction with AutoReduce

Assumptions such as quasi-steady state approximation (QSSA) and conservation laws are common in modeling of biological systems. [AutoReduce](https://github.com/ayush9pandey/autoReduce) is a Python package that can perform model reduction using QSSA and conservation laws on given SBML models. In addition to model reduction, AutoReduce can also perform sensitivity analysis and deterministic simulations for SBML models. 

This notebook gives a short introduction to AutoReduce by importing an SBML model and performing model reduction on the given model. 

# Detailed Chemical Reaction Network (CRN):

We have the SBML file called "gene_expression.xml" which models the expression of two genes according to the following reactions:

Transcription:

$ DNA + RNAP \leftrightharpoons C_1 $

$ C_1 \rightarrow mRNA + DNA + RNAP $

Translation: 

$ mRNA + Ribosome \leftrightharpoons C_2 $

$C_2 \rightarrow Protein + mRNA + Ribosome $

Degradation:

$ mRNA + RNAase \leftrightharpoons C_3 $

$ C_3 \rightarrow RNAase $

$ Protein \rightarrow \varnothing $

$ mRNA \rightarrow \varnothing $

The SBML file may be generated using any SBML model builder tool such as BioCRNpyler, COPASI, iBioSim, or others. We load this SBML file in AutoReduce and apply model reduction techniques:


# Package Installation 

If you do not have AutoReduce installed in your environment, uncomment the following line to install:

In [2]:
# !pip install autoreduce

## Check if it worked by importing the packages:

In [6]:
from autoreduce import *
import numpy as np
from sympy import symbols

# Load SBML in AutoReduce:

In [7]:
from autoreduce.converters import load_sbml
sys = load_sbml('gene_expression.xml', outputs = ['protein_X'])

## Explore the System object

AutoReduce stores models using the following syntax for each model:

$ \dot{x} = f(x, params)$ 

$ y = Cx$ (if linear)

or 

$ y = h(x, params)$

Here, $x$ are the state variables of the model (the list of species, if the model is a CRN), the params are the list of parameters (the reaction rates, if the model is a CRN). The outputs $y$ are the measured variables of the system.

In [8]:
sys.x

[dna_X,
 protein_RNAP_machinery,
 rna_X,
 complex_dna_X_protein_RNAP_machinery_,
 protein_Ribo_machinery,
 protein_X,
 complex_protein_Ribo_machinery_rna_X_,
 protein_RNAase_machinery,
 dna_cellular_processes,
 rna_cellular_processes,
 complex_dna_cellular_processes_protein_RNAP_machinery_,
 protein_cellular_processes,
 complex_protein_Ribo_machinery_rna_cellular_processes_,
 complex_protein_RNAase_machinery_rna_X_,
 complex_complex_protein_Ribo_machinery_rna_X__protein_RNAase_machinery_,
 complex_protein_RNAase_machinery_rna_cellular_processes_,
 complex_complex_protein_Ribo_machinery_rna_cellular_processes__protein_RNAase_machinery_]

In [9]:
# sys.f
sys.params

[kb__,
 ku_strong_,
 ktx_strong_,
 ku_weak_,
 ktl__,
 kb__transcription,
 ku__transcription,
 ktx__transcription,
 kb__translation,
 ku__translation,
 ktl__translation,
 ku__,
 kdeg__rna_degredation_mm,
 kdil__]

In [10]:
sys.x_init

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [11]:
sys.C

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]])

# Set up model reduction:

In [13]:
from autoreduce.utils import get_reducible
sys_reduce = get_reducible(sys)
sys_reduce.nstates_tol_min = 3 # Minimum number of states tbat you want in the reduced model

## Conservation Laws

In [14]:
G, P, C1, T, R, C2, E, C3, X = sys.x
conserved_quantities = [P + C1 - P_tot, R + C2 - R_tot, E + C3 - E_tot]
states_to_eliminate = [1,4,6]
f_cons = sys_reduce.set_conservation_laws(conserved_quantities, states_to_eliminate)

ValueError: too many values to unpack (expected 9)